Fine tuning GPT-2 for Generating philosophy using python package gpt-2-simple.

gpt-2-simple doesn't work in tensorflow 2, Tensorflow 1.x is used

Since I couln't find a way to use GPU in kaggle kernel with Tensorflow 1 versions, this notebook is designed for Google colab

In [ ]:
%tensorflow_version 1.x #this works only in google colab

Checking the tensorflow version

In [ ]:
import tensorflow as tf
tf.__version__

For training the model, choose a few philosophical books and convert them to text files.
After pre-processing, combine them all to a single text file for training

Pre-Processing

In [ ]:
import re
file_name = '/content/Beyond Good and Evil.txt'
file1 = open(file_name,"rt",encoding='utf8')
file = file1.readlines()
file1.close()
sentences = []
start = 0
for line in file:
  if line=='PREFACE\n': #to start the text from Preface
    start = 1
  if start!=1:
    continue 
  if line == 'FROM THE HEIGHTS\n':
    break
  if line.startswith('CHAPTER'):
    continue
  # Removing unwanted punctuations
  line = re.sub('--',' ',line)
  words = line.split()
  if len(words)<=2:
    continue
  # Removing numbers from the beginning of the line
  if words[0][0].isnumeric(): 
    newline = ' '.join(words[1:]) +'\n'
  else:
    newline = line
  sentences.append(newline)
# Combining the lines
training_data = ''.join(sentences)

Create a new text file and store the processed data in the new text file.
If you want to train on multiple text files, then combine all of them to the same text file

In [ ]:
try:
  newfile = open("/content/training_file.txt",'x')
  newfile.close()
except:
  print('Already Exists')
training_file = open("/content/training_file.txt",'w')
training_file.write(training_data)
training_file.close()

In [ ]:
!pip install gpt-2-simple

First the pre-trained model will be downloaded which will be fine-tuned and saved. Then the saved model can be again loaded to generate text.
The training will be done for 500 steps (you can continue training if the average loss keeps on decreasing by uploading the saved model and again training it) and a sample text will be generated every 100 steps.
On a GPU it takes around 10 minutes for 500 steps

In [ ]:
import gpt_2_simple as gpt2
import os

model_name = "124M" #any other model can also be chosen but 124M works fine here
if not os.path.isdir(os.path.join("models", model_name)):
  gpt2.download_gpt2(model_name=model_name)   # model will be saved into current directory under /models/124M/


file_name = '/content/training_file.txt'

    

sess = gpt2.start_tf_sess()
gpt2.finetune(sess,
              file_name,
              model_name=model_name,
              steps=500)

After the training is completed the model will be automatically saved.
Once trained, you can also use the model to generate philosphy using a prefix

In [ ]:
prefix = 'The purpose of life is'
gpt2.generate(sess, model_name='/content/checkpoint/run1',prefix = prefix)

The generated text output I got:

The purpose of life is to create God--to make him
the GREATEST EXPERIENCE OF LIFE make him the happiest man
and woman have ever known. And to ask the question: Did the happiest man
have a conception about life? Probably: but he who has the
question himself often enough, seldom forgets: "Sir, could not you have
done more for happiness by being more?" probably: he would have said
yes, but he knew not what. And to ask the question: Did the happiest man
have a conception about life? Probably: he is the exception, the rule.

Try not to use historical books as the generated text be offensive.
All kinds of books can be used here. Fictional books work great.
Hope this helps.